In [2]:
import pandas as pd
import numpy as np
import plotly as py
from plotly import tools
import plotly.graph_objs as go

from scipy import stats
import scipy.optimize
from scipy.optimize import OptimizeWarning
import warnings
import math
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

import mpl_finance
from matplotlib.dates import date2num
from datetime import datetime

In [3]:
df = pd.read_csv("EURUSD.csv")
df.columns = ["date", "open", "high", "low", "close", "AskVol"]

df.date = pd.to_datetime(df.date, format="%d.%m.%Y %H:%M:%S.%f") 
df = df.set_index(df.date)
df = df[["open", "high", "low", "close", "AskVol"]]
# drop market-close price per hour
df = df.drop_duplicates(keep=False)


df.head()

,open,high,low,close,AskVol
date,,,,,
2017-01-01 22:00:00,1.05236,1.05253,1.05221,1.05227,143.54
2017-01-01 23:00:00,1.05227,1.05426,1.05226,1.05282,253.14
2017-01-02 00:00:00,1.05283,1.05283,1.05268,1.05281,131.40
2017-01-02 01:00:00,1.05282,1.05286,1.05225,1.05240,273.94
2017-01-02 02:00:00,1.05239,1.05240,1.05164,1.05220,258.14


# Features Creation

### 1) Heiken Ashi - given the paper

In [6]:
class holder:
    1
    
def heikenashi(prices, periods):
    
    """
    ;param prices: dataframe of OHLC & volume data
    ;prama periods: periods for which to create the candles
    :return: heiken ashi OHLC candles
    
    """
    results = holder()
    
    dict = {}
    
    HAclose = prices[["open", "high", "low", "close"]].sum(axis=1)/4
    
    HAopen = HAclose.copy()
    HAopen.iloc[0] = HAclose.iloc[0]
    
    HAhigh = HAclose.copy()
    
    HAlow = HAclose.copy()
    
    for i in range(1, len(prices)):
        HAopen.iloc[i] = (HAopen.iloc[i-1] + HAclose.iloc[i-1])/2
        
        HAhigh.iloc[i] = np.array([prices.high.iloc[i], HAopen.iloc[i], HAclose.iloc[i]]).max()
        
        HAlow.iloc[i] = np.array([prices.low.iloc[i], HAopen.iloc[i], HAclose.iloc[i]]).min()
        
    df = pd.concat((HAopen, HAhigh, HAlow, HAclose), axis=1)
    df.columns = [["open", "high", "low", "close"]]
    
    
    #df.index = df.index.droplevel(0)
    
    dict[periods[0]] = df
    
    results.candles = dict
    
    return results



HAresults = heikenashi(df, [1])
HA = HAresults.candles[1]

ma = df.close.rolling(center=False, window=30).mean()

trace0 = go.Ohlc(x=df.index, open=df.open, high=df.high, low=df.low, close=df.close, name="Currency Quote")
trace1 = go.Scatter(x=df.index, y=ma)
trace2 = go.Ohlc(x=HA.index, open=HA.open, high=HA.high, low=HA.low, close=HA.close, name="Heiken Ashi")

data = [trace0, trace1, trace2]


fig = tools.make_subplots(rows=2, cols=1, shared_xaxes=True, shared_yaxes=True)
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)

py.offline.plot(fig, filename="ccc.html")

ValueError: 
    Invalid value of type 'pandas.core.frame.DataFrame' received for the 'close' property of ohlc
        Received value:                         close
date                         
2017-01-01 22:00:00  1.052342
2017-01-01 23:00:00  1.052903
2017-01-02 00:00:00  1.052787
2017-01-02 01:00:00  1.052583
2017-01-02 02:00:00  1.052157
2017-01-02 03:00:00  1.052183
2017-01-02 04:00:00  1.052080
2017-01-02 05:00:00  1.051723
2017-01-02 06:00:00  1.051560
2017-01-02 07:00:00  1.051762
2017-01-02 08:00:00  1.050310
2017-01-02 09:00:00  1.048717
2017-01-02 10:00:00  1.048447
2017-01-02 11:00:00  1.047993
2017-01-02 12:00:00  1.047250
2017-01-02 13:00:00  1.046605
2017-01-02 14:00:00  1.047065
2017-01-02 15:00:00  1.047138
2017-01-02 16:00:00  1.047305
2017-01-02 17:00:00  1.046762
2017-01-02 18:00:00  1.046435
2017-01-02 19:00:00  1.046215
2017-01-02 20:00:00  1.046037
2017-01-02 21:00:00  1.045687
2017-01-02 22:00:00  1.046218
2017-01-02 23:00:00  1.046270
2017-01-03 00:00:00  1.04

In [ ]:
# moving average
ma = df.close.rolling(center=False, window=30).mean()
# candlestick graph
trace0 = go.Ohlc(x=df.index, open=df.open, high=df.high, low=df.low, close=df.close, name="Currency Quote")
trace1 = go.Scatter(x=df.index, y=ma)
trace2 = go.Bar(x=df.index, y=df.volume)

data = [trace0, trace1, trace2]

In [ ]:
fig = tools.make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)

py.offline.plot(fig, filename="xxx.html")

In [ ]:
 HA.close